In [1]:
import librosa

In [11]:
data,sr= librosa.load('./commands/bongeszo/bongeszo--hangNormTempoMinusz5Szazalek-01.wav')
mfccs = librosa.feature.mfcc(data, sr=sr)

C:\Users\Laci\AppData\Local\Temp\ipykernel_9124\1721168264.py:2: FutureWarning: Pass y=[ 1.19364095e-05  1.78441944e-04  3.54877295e-04 ... -5.26354097e-06
  5.56010036e-06  0.00000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = librosa.feature.mfcc(data, sr=sr)


In [12]:
mfccs.shape

(20, 95)

In [2]:
import os
import librosa
import numpy as np

# Define constants
SAMPLE_RATE = 22050
WINDOW_SIZE = 0.1 # Window size in seconds
WINDOW_STRIDE = 0.05 # Window stride in seconds
N_MFCC = 20 # Number of MFCC coefficients to compute

# Define function to extract MFCCs from audio file
def extract_mfcc(file_path):
    # Load audio file
    signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)

    # Compute MFCCs using sliding window
    mfccs = []
    for i in range(0, len(signal) - int(WINDOW_SIZE * SAMPLE_RATE), int(WINDOW_STRIDE * SAMPLE_RATE)):
        window = signal[i:i + int(WINDOW_SIZE * SAMPLE_RATE)]
        mfcc = librosa.feature.mfcc(window, sr=SAMPLE_RATE, n_mfcc=N_MFCC)
        mfccs.append(mfcc)

    return mfccs

# Define function to load and extract MFCCs from multiple audio files
def load_data(data_dir):
    # Initialize lists to store data
    X = []
    y = []

    # Loop over subdirectories (each subdirectory represents a class)
    for i, sub_dir in enumerate(os.listdir(data_dir)):
        sub_dir_path = os.path.join(data_dir, sub_dir)

        # Loop over audio files in subdirectory
        for file_name in os.listdir(sub_dir_path):
            file_path = os.path.join(sub_dir_path, file_name)

            # Extract MFCCs from audio file
            mfccs = extract_mfcc(file_path)

            # Append MFCCs and label to data lists
            X.append(mfccs)
            y.append(i)

    # Convert data lists to numpy arrays
    X = np.array(X)
    y = np.array(y)

    return X, y


In [3]:
import keras
from keras.utils import to_categorical

In [ ]:
%%capture
# Load data from directory containing audio files
data_dir = './commands/'
X, y = load_data(data_dir)

# Print shape of data arrays
print(X.shape) # Shape should be (num_files, num_windows, num_mfcc, window_size_in_frames)
print(y.shape) # Shape should be (num_files,)

# Convert labels to one-hot encoded format
y = to_categorical(y, num_classes=11)


# Print shape of data arrays
print(X.shape) # Shape should be (num_files, num_windows, num_mfcc, window_size_in_frames)
print(y.shape) # Shape should be (num_files, num_classes)

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

In [21]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42, shuffle=True)

In [22]:
num_classes=11

In [24]:
model = Sequential()
model.add(LSTM(128,
               recurrent_dropout=0.1,
               return_sequences=True,
               input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(128,
               recurrent_dropout=0.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

IndexError: tuple index out of range